In [2]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 61.6 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
import gensim.downloader as api

In [4]:
print("📥 მონაცემების ჩამოტვირთვა...")
dataset = load_dataset("ag_news")
train_subset = dataset['train'].select(range(5000))  # 5000 ტრენინგისთვის
test_subset = dataset['test'].select(range(1000))    # 1000 ტესტირებისთვის

📥 მონაცემების ჩამოტვირთვა...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [5]:
train_subset

Dataset({
    features: ['text', 'label'],
    num_rows: 5000
})

In [6]:
test_subset

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})

In [7]:
train_texts = train_subset['text']
train_labels = train_subset['label']
test_texts = test_subset['text']
test_labels = test_subset['label']

# 2. Bag of Words (TF-IDF) მიდგომა
print("📊 TF-IDF ვექტორიზაცია...")
tfidf = TfidfVectorizer(max_features=2000, stop_words='english')
X_train_tfidf = tfidf.fit_transform(train_texts)
X_test_tfidf = tfidf.transform(test_texts)

📊 TF-IDF ვექტორიზაცია...


In [8]:
# 3. GloVe Embeddings მიდგომა
print("🌍 GloVe მოდელის ჩამოტვირთვა (glove-wiki-gigaword-100)...")
glove_model = api.load("glove-wiki-gigaword-100")

def get_doc_vector(text):
    words = text.lower().split()
    vectors = [glove_model[w] for w in words if w in glove_model]
    if not vectors:
        return np.zeros(100)
    return np.mean(vectors, axis=0)

🌍 GloVe მოდელის ჩამოტვირთვა (glove-wiki-gigaword-100)...
[==================================================] 100.0% 128.1/128.1MB downloaded


In [9]:
print("🧪 სტატიების ვექტორიზაცია (Embedding averaging)...")
X_train_glove = np.array([get_doc_vector(t) for t in train_texts])
X_test_glove = np.array([get_doc_vector(t) for t in test_texts])

🧪 სტატიების ვექტორიზაცია (Embedding averaging)...


In [10]:
# 4. კლასიფიკაცია KNN-ის გამოყენებით
def evaluate_knn(X_train, y_train, X_test, y_test, title):
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train, y_train)
    preds = knn.predict(X_test)
    print(f"\n--- {title} ---")
    print(f"Accuracy: {accuracy_score(y_test, preds):.4f}")
    print(classification_report(y_test, preds, target_names=dataset['train'].features['label'].names))

evaluate_knn(X_train_tfidf, train_labels, X_test_tfidf, test_labels, "Bag of Words (TF-IDF)")
evaluate_knn(X_train_glove, train_labels, X_test_glove, test_labels, "GloVe Embeddings (Average)")


--- Bag of Words (TF-IDF) ---
Accuracy: 0.7350
              precision    recall  f1-score   support

       World       0.85      0.72      0.78       268
      Sports       0.91      0.70      0.79       274
    Business       0.82      0.56      0.66       205
    Sci/Tech       0.56      0.93      0.70       253

    accuracy                           0.73      1000
   macro avg       0.78      0.73      0.73      1000
weighted avg       0.78      0.73      0.74      1000


--- GloVe Embeddings (Average) ---
Accuracy: 0.8320
              precision    recall  f1-score   support

       World       0.83      0.83      0.83       268
      Sports       0.91      0.95      0.93       274
    Business       0.77      0.68      0.73       205
    Sci/Tech       0.80      0.83      0.81       253

    accuracy                           0.83      1000
   macro avg       0.83      0.82      0.82      1000
weighted avg       0.83      0.83      0.83      1000



In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

# შევქმნათ მოდელების სია შესადარებლად
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Linear SVM": LinearSVC(dual=False),
    "Random Forest": RandomForestClassifier(n_estimators=100)
}

In [12]:
results_new = []

print("🚀 ახალი ალგორითმების ტესტირება GloVe-ის ვექტორებზე...")

for name, clf in models.items():
    # დატრენინგება GloVe-ზე (რომელმაც წინა ტესტში უკეთესი შედეგი აჩვენა)
    clf.fit(X_train_glove, train_labels)
    preds = clf.predict(X_test_glove)

    acc = accuracy_score(test_labels, preds)
    results_new.append({"Model": name, "Accuracy": acc})

    print(f"✅ {name}: Accuracy = {acc:.4f}")

🚀 ახალი ალგორითმების ტესტირება GloVe-ის ვექტორებზე...
✅ Logistic Regression: Accuracy = 0.8670
✅ Linear SVM: Accuracy = 0.8750
✅ Random Forest: Accuracy = 0.8510


In [13]:
import pandas as pd
df_new = pd.DataFrame(results_new)
print("\n--- შედარებითი ცხრილი (GloVe Embeddings) ---")
print(df_new.to_markdown(index=False))


--- შედარებითი ცხრილი (GloVe Embeddings) ---
| Model               |   Accuracy |
|:--------------------|-----------:|
| Logistic Regression |      0.867 |
| Linear SVM          |      0.875 |
| Random Forest       |      0.851 |
